<a href="https://colab.research.google.com/github/higgsbelly/bitGANs-Generator/blob/main/Color_Shift_GAN_Training_Data_Generator_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Another notebook to help on your journey to making your own BitGANs, but a generally usefule tool for genrative pixel art!

To start, you need your layers organized in the follwoing way. 


*   assets
 *   0-background
 *   1-texture
 *   2-body
 *   3-face
 *   4-head
 *   5-overlay

The number represents the order you would like that layer pasted to the canvas. 

For an example see: https://drive.google.com/drive/folders/10CwNXNfTMd5wsLMnNGFr2lDzgESUOdvD?usp=sharing

Next you will need to specify how many training images you would like to create. This is done by changing the number next to the **Num** variable below. ie **Num = 20** to make 20 training images. 

You will need to create a second folder **Train_Data** where your training images will be saved. Both folders **assets** and **Train_Data** need to be in your main drive. 

Once you are happy with your training inmages, take them over to BitGAN Generator and see how turn out!

This document is still being editted...





Mount your drive, grant permissions. Make a copy if you plan to make edits.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive/
Note: using Google CoLab


All you need to edit below is **Num = (number of images you want), px = (size of images you want, needs to be multiple of 2)**, and **dx (what half the distance between the center of the eyes is, your left right shift)** 

The code below will only build your images. To build your images with a spectrum, execute the code block below this one.

In [2]:
import random
import os
import shutil

from PIL import Image

These are the color palettes, which we set up in advance.

In [3]:
# colors by name
MAGENTA   = [255,1,191] #ff01bf
PINK      = [255,1,98] #ff0162
RED       = [255,1,1] #ff0101
ORANGE    = [255,94,1] #ff5e01
AMBER     = [255,191,1] #ffc001 
YELLOW    = [255,1,1] #ffff01
LIME      = [128,255,1] #80ff01
GREEN     = [1,255,1] #01ff01
MINT      = [1,255,157] #01ff9d
TURQUOISE = [1,255,255] #01ffff
CYAN      = [1,161,255] #01a1ff
BLUE      = [1,1,255] #0101ff
PURPLE    = [128,1,255] #80019b
VIOLET    = [221,1,255] #dd01ff
WHITE     = [254,254,254] #fefefe
GRAY      = [128,128,128] #888888
BLACK     = [1,1,1] #010101

# create arrays of color for each layer
background_color = [ CYAN, MAGENTA, YELLOW ]
texture_color =    [ MINT, TURQUOISE, CYAN, BLUE, PURPLE ]
body_color =       [ GREEN, MINT, TURQUOISE, CYAN, BLUE, PURPLE, VIOLET ] 
head_color =       [ MAGENTA, AMBER, LIME, TURQUOISE, VIOLET ]
face_color =       [ MAGENTA, RED, LIME, GREEN, TURQUOISE, BLUE, PURPLE, VIOLET ]
overlay_color =    [ MAGENTA, PINK, RED, AMBER, ORANGE, YELLOW, LIME, GREEN, MINT, TURQUOISE, CYAN, BLUE, PURPLE, VIOLET ]

# NOTE: We will assume the assets (and their directories) are in this order.
# When naming folders with assets, previx the folder names with 0, 1, 2, etc.
# to guarantee their order.
asset_list = [background_color, texture_color, body_color, head_color, face_color, overlay_color]

# this function changes the primary color, making it "ultralight"
def makeUltralight(rgb):
  uRed = (rgb[0])+((255-rgb[0])*0.9)
  uGreen = (rgb[1])+((255-rgb[1])*0.9)
  uBlue = (rgb[2])+((255-rgb[2])*0.9)
  return [uRed, uGreen, uBlue]

# this function changes the primary color, making it "muted"
def makeMuted(rgb):
  mRed = ((rgb[0])+((255-rgb[0])*0.5))*0.5
  mGreen = ((rgb[1])+((255-rgb[1])*0.5))*0.5
  mBlue = ((rgb[2])+((255-rgb[2])*0.5))*0.5
  return [mRed, mGreen, mBlue]

# this function changes the primary color, making it "light"
def makeLight(rgb):
  lRed = (rgb[0])+((255-rgb[0])*0.5)
  lGreen = (rgb[1])+((255-rgb[1])*0.5)
  lBlue = (rgb[2])+((255-rgb[2])*0.5)
  return [lRed, lGreen, lBlue]

# this function changes the primary color, making it "dark"
def makeDark(rgb):
  dRed = (rgb[0])*0.66
  dGreen = (rgb[1])*0.66
  dBlue = (rgb[2])*0.66
  return [dRed, dGreen, dBlue]

In [4]:
def color_shift(color_old, color_new):
  # In each layer, all pixels that are pure black, red, green, and blue will be 
  # replaced by different versions of the default palette.

  # white pixels (255,255,255) will be replaced by an ULTRALIGHT version of the color
  if color_old == (255, 255, 255):
    color_new = makeUltralight(color_new)

  # black pixels (0,0,0) will be replaced by a MUTED version of the color
  if color_old == (0, 0, 0):
    color_new = makeMuted(color_new)

  # green pixels (0,255,0) will be replaced by a LIGHT version of the color
  if color_old == (0, 255, 0):
    color_new = makeLight(color_new)

  # red pixels (255,0,0) will be replaced by the DEFAULT version of the color
  if color_old == (255, 0, 0):
    color_new = color_new
  
  # blue pixels (0,0,255) will be replaced by a DARK version of the color
  if color_old == (0, 0, 255):
    color_new = makeDark(color_new)

  color_new = [round(num) for num in color_new]
  return tuple(color_new)

In [5]:
def color_shift_img(img, colors):
  pixdata = img.load()

  colors_new = dict()
  for y in range(32):
    for x in range(32):
      color_old = pixdata[x, y]
      
      # if transparent, skip
      if color_old[3] == 0:
        continue
      
      # use a dictionary to track colors
      if color_old not in colors_new:
        colors_new[color_old] = random.choice(colors)

      pixdata[x, y] = color_shift(color_old[0:3], colors_new[color_old])

  return(img)

In [6]:
def make_image(imput_dir, d, px, d_x, b_x):
  img = Image.new('RGBA', (32, 32))
  shift_index = [0,1,2,2,2,2]
  j = 0
  d_x = d_x*random.randint(-1,1)
  b_x = b_x*random.randint(-1,1)

  for key, val in d.items():
    tmp = Image.open(os.path.join(input_dir, key, random.choice(val))).convert('RGBA')

    # this might not always work
    i = int(key[0])    
    tmp = color_shift_img(tmp, asset_list[i])

    if shift_index[j] == 0:
      img.paste(tmp, (0,0), tmp)
    elif shift_index[j] == 1:
      img.paste(tmp, (b_x,0), tmp)
    else:
      img.paste(tmp, (d_x,0), tmp)
    j += 1
    
  img = img.resize((px, px), resample = 0)
  return(img)

In [8]:
# where your not so shifted training data lives
input_dir = '/content/drive/MyDrive/bitgan_folders/assets/' 

# where you want to save the shifted pngs
output_dir = '/content/drive/MyDrive/bitgan_folders/output/' 

Let's query the folder to see what assets exist. They are stored in a dictionary and will be accessed in order (I hope).

In [9]:
onlyfolders = [f for f in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, f))] 
onlyfolders = sorted(onlyfolders)
N = len(onlyfolders)

d = {}
d = {key: None for key in onlyfolders}

for key in d:
  temp_path = os.path.join(input_dir, key)
  d[key] = [f for f in os.listdir(temp_path) if os.path.isfile(os.path.join(temp_path, f))]

In [10]:
n = 1000   # Number of images you want
px = 512 # Size you want the ouput image to be
dx = 4 #Distance between center of eyes divided by two
bx = 2 #Controls Background Shift

# WARNING: delete and create the output directory
shutil.rmtree(output_dir, ignore_errors=True)
os.mkdir(output_dir)

for i in range(n):
  img = make_image(input_dir, d, px, dx, bx)
  img.save(os.path.join(output_dir, f'{i:03}.png'))